In [1]:
import pandas as pd

# PVF Data

In [2]:
df = pd.read_csv("../data/parsed_logs_fi.csv")

In [3]:
configs = dict(
    # Baseline C100
    BaseC100="c100_res44_test_01_bn-relu_base.yaml",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="c100_res44_test_02_relu6-bn.yaml",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C100="c100_res44_test_02_bn-relu6_base.yaml",
    BnRelu6C100="c100_res44_test_02_bn-relu6.yaml",
    # Order inversion + nan filter + Relu6
    OrderINanRelu6C100="c100_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C100
    GeluNanC100="c100_res44_test_02_gelu6_nans.yaml",

    # Baseline C10
    BaseC10="c10_res44_test_01_bn-relu_base.yaml",
    # "c10_res44_test_02_bn-relu6.yaml",
    BnRelu6C10="c10_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C10
    OrderInvC10="c10_res44_test_02_relu6-bn.yaml",
    # "c10_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C10="c10_res44_test_02_bn-relu6_base.yaml",
    OrderINanRelu6C10="c10_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C10
    GeluNanC10="c10_res44_test_02_gelu6_nans.yaml",

    MobileNetC100="cifar100_mobilenetv2_x1_4.yaml",
    MobileNetC10="cifar10_mobilenetv2_x1_4.yaml",

    Resnet50ImagenetBase="imagenet1k_v2_base",
    TinyImagenetBase="tiny_res56_bn-relu_base",
    TinyImagenetAware="tiny_res56_relu6-bn_aware",
    TinyImagenetAwareNans="tiny_res56_relu6-bn_aware_nans",
)
configs_inv = {v.replace(".yaml", ""): k for k, v in configs.items()}

In [4]:
NEW_NAMES = dict(
    # Baseline C100
    BaseC100="Base",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="OrdInv",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C100="BaseR6",
    BnRelu6C100="BnR6",
    # Order inversion + nan filter + Relu6
    OrderINanRelu6C100="OrdInvNanR6",
    # Gelu and nan C100
    GeluNanC100="Gelu6Nan",
    TinyImagenetBase="TinyImagenetBase",
    TinyImagenetAware="TinyImagenetAware",
    TinyImagenetAwareNans="TinyImagenetAwareNans",
)
NEW_NAMES.update({k.replace("C100", "C10"): v for k, v in NEW_NAMES.items()})
NEW_NAMES["Resnet50ImagenetBase"] = "Resnet50ImagenetBase"

In [5]:
configs_df = pd.DataFrame([configs]).T.reset_index().rename(columns={0: "YAML", "index": "config"})

In [6]:
configs_df

,config,YAML
0,BaseC100,c100_res44_test_01_bn-relu_base.yaml
1,OrderInvC100,c100_res44_test_02_relu6-bn.yaml
2,BaseBnRelu6C100,c100_res44_test_02_bn-relu6_base.yaml
3,BnRelu6C100,c100_res44_test_02_bn-relu6.yaml
4,OrderINanRelu6C100,c100_res44_test_02_relu6-bn_nanfilter.yaml
5,GeluNanC100,c100_res44_test_02_gelu6_nans.yaml
6,BaseC10,c10_res44_test_01_bn-relu_base.yaml
7,BnRelu6C10,c10_res44_test_02_bn-relu6.yaml
8,OrderInvC10,c10_res44_test_02_relu6-bn.yaml
9,BaseBnRelu6C10,c10_res44_test_02_bn-relu6_base.yaml


In [7]:
df["config"] = df["config"].apply(lambda x: configs_inv[x])

In [8]:
df["count"] = 1

In [9]:
to_save = df.groupby(["hostname", "config", "fi_model"]).sum()[["sdc", "critical_sdc", "count"]].reset_index()

In [10]:
to_save

,hostname,config,fi_model,sdc,critical_sdc,count
0,carol-titanv,TinyImagenetAware,0,31,0,31
1,carol-titanv,TinyImagenetAware,2,144,5,144
2,carol-titanv,TinyImagenetAware,3,65,2,65
3,carol-titanv,TinyImagenetAware,4,244,33,244
4,carol-titanv,TinyImagenetAware,5,216,2,216
...,...,...,...,...,...,...
100,caroltitantn,OrderInvC100,0,152,0,152
101,caroltitantn,OrderInvC100,2,224,8,224
102,caroltitantn,OrderInvC100,3,193,0,193
103,caroltitantn,OrderInvC100,4,308,37,308


In [11]:
FM_NAMES = dict(FLIP_SINGLE_BIT="Single Bit", ZERO_VALUE="Zero Val", WARP_ZERO_VALUE="Warp Z.V.",
                RANDOM_VALUE="Rand Val", WARP_RANDOM_VALUE="Warp R. V.")
FM_IDS = dict(FLIP_SINGLE_BIT=0, RANDOM_VALUE=2, ZERO_VALUE=3, WARP_RANDOM_VALUE=4, WARP_ZERO_VALUE=5)
FM_ID_TO_NAMES = {fmv: FM_NAMES[fmk] for fmk, fmv in FM_IDS.items()}

In [12]:
def set_dataset(x):
    if "C100" in x:
        return "Cifar 100"
    elif "Imagenet" in x:
        return "Imagenet"
    elif "tinyimagenet" in x.lower():
        return "Tiny Imagenet"
    else:
        return "Cifar 10"


In [13]:
to_save["dataset"] = to_save["config"].apply(set_dataset)
to_save["config"] = to_save["config"].apply(lambda x: NEW_NAMES[x])
to_save["fi_model"] = to_save["fi_model"].apply(lambda x: FM_ID_TO_NAMES[x])

In [14]:
to_save

,hostname,config,fi_model,sdc,critical_sdc,count,dataset
0,carol-titanv,TinyImagenetAware,Single Bit,31,0,31,Imagenet
1,carol-titanv,TinyImagenetAware,Rand Val,144,5,144,Imagenet
2,carol-titanv,TinyImagenetAware,Zero Val,65,2,65,Imagenet
3,carol-titanv,TinyImagenetAware,Warp R. V.,244,33,244,Imagenet
4,carol-titanv,TinyImagenetAware,Warp Z.V.,216,2,216,Imagenet
...,...,...,...,...,...,...,...
100,caroltitantn,OrdInv,Single Bit,152,0,152,Cifar 100
101,caroltitantn,OrdInv,Rand Val,224,8,224,Cifar 100
102,caroltitantn,OrdInv,Zero Val,193,0,193,Cifar 100
103,caroltitantn,OrdInv,Warp R. V.,308,37,308,Cifar 100


In [15]:
to_save_tail = to_save.groupby(["hostname", "dataset", "config"]).sum()

In [16]:
with pd.ExcelWriter("/home/fernando/git_research/tetc_2023_diehardnet/data/avf_per_fm_tmp.xlsx") as writer:
    to_save.groupby(["hostname", "dataset", "config"]).sum()[["sdc", "critical_sdc"]].to_excel(writer, sheet_name="avf_per_fm")
    to_save.groupby(["dataset", "config"]).sum()[["sdc", "critical_sdc"]].to_excel(writer, sheet_name="avf_full")
    

In [17]:
to_save_tail["fi_model"] = "Overall"

In [18]:
to_save_gp = pd.concat([to_save, to_save_tail])

In [19]:
to_save_gp = to_save_gp.groupby(["hostname", "dataset", "fi_model", "config"]).sum()

In [20]:
to_save_gp

sdc  critical_sdc  \
hostname     dataset   fi_model   config                                     
carol-titanv Imagenet  Rand Val   TinyImagenetAware      144             5   
                                  TinyImagenetAwareNans  159             4   
                                  TinyImagenetBase       141            72   
                       Single Bit TinyImagenetAware       31             0   
                                  TinyImagenetAwareNans   22             1   
...                                                      ...           ...   
caroltitantn Cifar 100 Zero Val   BaseR6                 205             0   
                                  BnR6                   183             0   
                                  Gelu6Nan               290             0   
                                  OrdInv                 193             0   
                                  OrdInvNanR6            200             0   

                                                         count  
hostname     dataset   fi_model   config                        
carol-titanv Imagenet  Rand Val   TinyImagenetAware        144  
                                  TinyImagenetAwareNans    159  
                                  TinyImagenetBase         141  
                       Single Bit TinyImagenetAware         31  
                                  TinyImagenetAwareNans     22  
...                                                        ...  
caroltitantn Cifar 100 Zero Val   BaseR6                   205  
                                  BnR6                     183  
                                  Gelu6Nan                 290  
                                  OrdInv                   193  
                                  OrdInvNanR6              200  

[105 rows x 3 columns]

In [21]:
to_save_gp["avf sdc"] = (to_save_gp["sdc"] - to_save_gp["critical_sdc"]) / to_save_gp["count"]
to_save_gp["avf critical sdc"] = to_save_gp["critical_sdc"] / to_save_gp["count"]

# Data on NaNs on fault simulation

In [22]:
add_info_df = pd.read_csv("../data/additional_data_fi.csv")
add_info_df["nan"] = add_info_df["nan"].astype(int)
add_info_df["inf"] = add_info_df["inf"].astype(int)
add_info_df["fi_model"] = add_info_df["fi_model"].apply(lambda x: FM_ID_TO_NAMES[x])
add_info_df["dataset"] = add_info_df["config"].apply(set_dataset)
add_info_df["config"] = add_info_df["config"].apply(lambda x: configs_inv[x.replace("diehardnet_", "")])
add_info_df["count"] = 1
add_info_df = add_info_df[~add_info_df["config"].str.contains("Gelu")]

In [23]:
add_info_df

,config,fault_it,fi_model,group,nan,inf,sdc,critical_sdc,there_is_nan_on_output,there_is_inf_on_output,dataset,count
0,BaseC10,1,Single Bit,1,0,0,0.0,0.0,0.0,0.0,Cifar 10,1
1,BaseC10,2,Single Bit,1,0,0,0.0,0.0,0.0,0.0,Cifar 10,1
2,BaseC10,3,Single Bit,1,0,0,1.0,0.0,0.0,0.0,Cifar 10,1
3,BaseC10,4,Single Bit,1,0,0,1.0,0.0,0.0,0.0,Cifar 10,1
4,BaseC10,5,Single Bit,1,0,0,0.0,0.0,0.0,0.0,Cifar 10,1
...,...,...,...,...,...,...,...,...,...,...,...,...
41995,TinyImagenetAwareNans,396,Warp Z.V.,1,0,0,0.0,0.0,0.0,0.0,Cifar 10,1
41996,TinyImagenetAwareNans,397,Warp Z.V.,1,0,0,0.0,0.0,0.0,0.0,Cifar 10,1
41997,TinyImagenetAwareNans,398,Warp Z.V.,1,0,0,0.0,0.0,0.0,0.0,Cifar 10,1
41998,TinyImagenetAwareNans,399,Warp Z.V.,1,0,0,1.0,1.0,0.0,0.0,Cifar 10,1


In [24]:
pct = add_info_df.groupby(["config", "critical_sdc"]).sum()[["nan", "count"]]
pct["nan%"] = pct["nan"] / pct["count"] * 100
# pct["inf%"]  = pct["inf"] / pct["count"] * 100


In [25]:
pct.groupby("critical_sdc").agg({"count": "sum", "nan%": "mean"})

,count,nan%
critical_sdc,,
0.0,34743,3.347987
1.0,1257,14.554945


In [38]:
add_info_df["nan"].sum() / add_info_df["count"].sum() * 100

3.5527777777777776

In [26]:
pct.groupby("critical_sdc").agg({"count": "sum", "nan%": "mean"})["count"].sum()


36000

In [27]:
pct.reset_index().drop(colaumns=["critical_sdc"]).groupby("config").mean()

,nan,count,nan%
config,,,
BaseBnRelu6C10,34.5,1000.0,14.970613
BaseBnRelu6C100,73.0,2000.0,9.825789
BaseC10,40.0,1000.0,4.926293
BaseC100,76.5,2000.0,3.918974
BnRelu6C10,28.0,1000.0,14.968691
BnRelu6C100,70.5,2000.0,6.350192
OrderINanRelu6C10,28.5,1000.0,1.426426
OrderINanRelu6C100,73.5,2000.0,3.234612
OrderInvC10,31.0,1000.0,30.026521


In [28]:
pct

nan  count       nan%
config                critical_sdc                       
BaseBnRelu6C10        0.0            65   1985   3.274559
                      1.0             4     15  26.666667
BaseBnRelu6C100       0.0           132   3914   3.372509
                      1.0            14     86  16.279070
BaseC10               0.0            67   1787   3.749301
                      1.0            13    213   6.103286
BaseC100              0.0           136   3579   3.799944
                      1.0            17    421   4.038005
BnRelu6C10            0.0            53   1989   2.664656
                      1.0             3     11  27.272727
BnRelu6C100           0.0           133   3914   3.398058
                      1.0             8     86   9.302326
OrderINanRelu6C10     0.0            57   1998   2.852853
                      1.0             0      2   0.000000
OrderINanRelu6C100    0.0           145   3928   3.691446
                      1.0             2     72   2.777778
OrderInvC10           0.0            58   1993   2.910186
                      1.0             4      7  57.142857
OrderInvC100          0.0           145   3909   3.709389
                      1.0            16     91  17.582418
TinyImagenetAware     0.0            63   1958   3.217569
                      1.0             6     42  14.285714
TinyImagenetAwareNans 0.0            63   1975   3.189873
                      1.0             1     25   4.000000
TinyImagenetBase      0.0            67   1814   3.693495
                      1.0             7    186   3.763441